In [1]:
import json
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
keywords = pd.read_excel('keywords1.xlsx',header=None)
keywords

,0
0,female+genetalia+development+proteins
1,molecular+mechanisms+disorders+sex+development
2,molecular+mechanisms+gonadal+development
3,23+XX+DSD+molecular+mechanisms
4,23+XY+DSD+molecular+mechanisms


In [3]:
def url_builder(term,size,page):
    URL = 'https://pubmed.ncbi.nlm.nih.gov/?term={}&format=abstract&size={}&page={}'.format(\
                                                                                           term,
                                                                                           size,
                                                                                           page)
    return URL

In [4]:
def get_html(URL):
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")
    html = soup.find_all('div', {'class': "results-article"})
    return html

In [5]:
def get_data(result):
    title = result.find('h1').text.replace('\n','').strip()
    try:
        doi = result.find('span',{'class':"citation-doi"}).text.strip()
    except:
        doi = 'NA'
    pubmedId = result.find('strong',{'class':"current-id", 'title':"PubMed ID"}).text
    a = result.find('div',{'class':"abstract"}).find_all('p')
    try:
        abstract = ''
        for i in a:
            abstract = abstract + i.text.strip() + ' '
            abstract = abstract.replace('\n','')
    except:
        abstract = 'NA'
    link = 'https://pubmed.ncbi.nlm.nih.gov/{}/'.format(pubmedId)
    authors = []
    for i in result.find_all('a',{'class':'full-name'}):
        authors.append(i.text)
    authors = list(dict.fromkeys(authors))
    try:
        date = result.find('span',{'class':'cit'}).text
        date = date.split(';')[0]
    except:
        date = 'NA'
    affiliations = []
    try:
        for x in result.find('ul',{'class':'item-list'}).find_all('li'):
            affiliations.append(x.text)
        affiliations
    except:
        affiliations = []
    data = [link,title,date,pubmedId,authors,abstract,affiliations,doi]
    return data

In [6]:
def save_page(html,size,page_n,term):
    page = []
    for i in range(size):
        data = get_data(html[i])
        page.append(data)
    page = pd.DataFrame(page)
    page.to_excel('{}_page_{}.xlsx'.format(term, page_n))    

In [7]:
for keyword in keywords[0]:
    print(keyword)
    URL = 'https://pubmed.ncbi.nlm.nih.gov/?term={}'.format(keyword)
    response = requests.get(URL)
    soup = BeautifulSoup(response.text, "html.parser")
    n_results = soup.find('div', {'class':'results-amount'}).find('span').text
    n_results = n_results.replace(',','')
    print(n_results)
    del(soup)
    npages = int(n_results)//200 +1
    print(npages)
    if int(n_results) < 200:
        URL = url_builder(keyword,200,1)
        html = get_html(URL)
        save_page(html,int(n_results),1,term=keyword)
    else:    
        for i in range(npages-1):
            URL = url_builder(keyword,200,i+1)
            html = get_html(URL)
            save_page(html,200,i+1,term=keyword)
        URL = url_builder(keyword,200,i+2)
        html = get_html(URL)
        save_page(html,int(n_results)%200,i+2,term=keyword)

female+genetalia+development+proteins
23786
119
molecular+mechanisms+disorders+sex+development
1211
7
molecular+mechanisms+gonadal+development
2555
13
23+XX+DSD+molecular+mechanisms
2
1
23+XY+DSD+molecular+mechanisms
2
1
